# DeepStart Data Genertion

This program creates training data for deepstar usng the dijkstra algorithm on a randomly generated perlin noise heightmap.

## Import Python Modules

In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import loader, os, io, random, noise, math, time

from PIL import Image
from Dijkstra import *

## Data Generation Options

#### Data settings
**Shape**: Image dimensions in pixels.<br>
**Output**: Output folder path for images.<br>
**Images**: How many images to generate.<br>

#### Noise settings
**Scale**: Noise scale.<br>
**Octaves**: How many times to apply the noise.<br>
**Persistence**: How much the noise amplitue changes for each octave.<br>
**Lacunarity**: How much the noise frequency changes for each octave.

In [ ]:
shape = (256, 256)
output = "data"
images = 1

scale = 100.0
octaves = 6
persistence = 0.5
lacunarity = 2.0

## Data Generation
### Helper Function

#### Get Height

Calculates the perlin noise height at a given point (i, j).<br>
**Return**: A float value between -1 and 1

In [ ]:
def get_height(i, j):
    return noise.pnoise2(i/scale, j/scale, octaves=octaves, persistence=persistence, lacunarity=lacunarity, repeatx=shape[0], repeaty=shape[1], base=0)

#### Generate Map

Calculates noise values for the entire grid with the dimensions given in the shape setting. The heightvalues are normalized to between 0 and 1.<br>
**Return**: A grid of dimensions (shape.x, shape.y) that contains normalized perline height noise.

In [ ]:
def generate_map():
    grid = []
    for i in range(shape[0]):
        grid.append([])
        for j in range(shape[1]):
            n = get_height(i, j)
            n += 1
            n /= 2
            
            grid[i].append(Node(n, (i, j)))
            
    return grid

#### Get Node

Get node at point.<br>
**Return**: Node at point give.

In [ ]:
def get_node(world, point):
    return world[point[0]][point[1]]

#### Get Random Points

Calculates a random start and stop inside the shape given in the settings.<br>
**Return**: Start and End point (Not node).

In [ ]:
def get_random_points():
    start = (random.randrange(0, shape[1]), random.randrange(0, shape[1]))
    end = (random.randrange(0, shape[1]), random.randrange(0, shape[1]))
    
    return start, end

#### Create Image

Create a training image for DeepStar. The R axis contains the heightdata from the grid, G and B axies contains the start and stop point respectivly.<br>
**Return**: RGB training image fro DeepStar.

In [ ]:
def create_image(grid, start, end):
    im = Image.new("RGB", shape, "#00FF00")
    pixels = im.load()
    
    sx, sy = start
    ex, ey = end
    
    im.paste((0, 255, 0), (sx - 1, sy - 1, sx, sy))
    im.paste((0, 0, 255), (ex - 1, ey - 1, ex, ey))
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            intNoise = int(grid[i][j].value * 255)
            pixels[i, j] = (intNoise, 0, 0)
            
    return im

### Implementation

Loop for creating the training data

In [ ]:
start, end = get_random_points()
prev_progress = 0

print(f'Starting data generation of {images} image{("s" if images != 1 else "")}')
for image in range(images):
    world = generate_map()
    start_node = get_node(world, start)
    end_node = get_node(world, end)
    
    path = dijkstra(start_node, end_node, world)
    if path is None:
        continue
    
    midpoint = get_path_midpoint(path)
    mx, my = midpoint.point
    
    im = create_image(world, start, end)
    im.save(output + "{0}_{1}.png".format(mx, my, image))

    progress = round(((image + 1) / images))
    if (progress != prev_progress):
        print(f'{progress}%')
        prev_progress = progress

    start, end = get_random_points()